# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_df = pd.read_csv('clean_city_df.csv')
city_df

,City,Lat,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,Qaanaaq,77.4840,-69.3632,38.89,79,71,2.66,GL
1,Carnarvon,-24.8667,113.6333,48.27,38,66,10.36,AU
2,Kavieng,-2.5744,150.7967,81.82,77,100,10.02,PG
3,Butaritari,3.0707,172.7902,81.82,78,100,12.77,KI
4,Half Moon Bay,37.4636,-122.4286,83.59,71,75,1.01,US
...,...,...,...,...,...,...,...,...
570,Andra,62.5147,65.8878,60.03,52,100,6.17,RU
571,Carballo,43.2130,-8.6910,80.02,80,7,5.55,ES
572,Klichka,50.4457,117.9960,57.97,91,39,6.22,RU
573,Orange Cove,36.6244,-119.3137,91.85,42,1,4.61,US


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
gmaps.configure(api_key=g_key) # you have to replace the value YOUR_API_KEY by the key generated in the step 3.


heat_df = city_df[['Lat','Longitude']]
records = heat_df.to_records(index=False)
result = list(records)
locations = result
fig = gmaps.figure()
heat_lay = gmaps.heatmap_layer(locations=city_df[['Lat','Longitude']], weights= city_df['Humidity'],dissipating=False)

fig.add_layer(heat_lay)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
hotel_list = []

for index, row in city_df.iterrows():
    city_lat = str(row['Lat'])
    city_log = str(row['Longitude'])
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=" + city_lat+ "," + city_log + "&radius=5000&type=hotel&key=" + g_key
    r = requests.get(url)
    x = r.json()
    y = x['results']
    if not y:
        hotel_list.append("No Hotel Near By")
    else: 
        hotel_list.append(y[0]['name'])

city_df['Hotel'] = hotel_list

In [17]:
hotel_df = city_df[['City','Lat','Longitude','Hotel','Country']]
hotel_df

,City,Lat,Longitude,Hotel,Country
0,Qaanaaq,77.4840,-69.3632,Qaanaaq,GL
1,Carnarvon,-24.8667,113.6333,Carnarvon,AU
2,Kavieng,-2.5744,150.7967,Kaewieng,PG
3,Butaritari,3.0707,172.7902,Ukiangang,KI
4,Half Moon Bay,37.4636,-122.4286,Half Moon Bay,US
...,...,...,...,...,...
570,Andra,62.5147,65.8878,Andra,RU
571,Carballo,43.2130,-8.6910,Carballo,ES
572,Klichka,50.4457,117.9960,Klichka,RU
573,Orange Cove,36.6244,-119.3137,Orange Cove,US


In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Longitude"]]

In [20]:
marker_locations = hotel_df[['Lat','Longitude']]
markers = gmaps.marker_layer(marker_locations)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))